In [ ]:
import pandas as pd
import numpy as np
import math
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import scipy
from scipy import optimize
from IPython.display import display, Math, Latex
import yfinance as yf

# Parent folder
import pathlib
from pathlib import Path  
pf = pathlib.Path().resolve() # Points to parent folder containing notebook and data, eg: 'C:/Users/Carla/Dropbox/Uni/10. Semester/Dynamic Programming/Term paper'

from tqdm import tqdm # for-loop progress bar

# Temporarily supress warnings
import warnings
warnings.filterwarnings("ignore") 


# Load data

## Bub estimates function

In [ ]:
def get_bub_estimates(name, volume_weighted, vlow, ma, start_dates, end_dates):
    # Read and format data
    filepath= Path(f'{pf}/data/output/bub_interval_estimates_final_{start_dates[0][0:4]}-{end_dates[-1][0:4]}_VW{volume_weighted}_{name}_all.csv')
    df_main = pd.read_csv(filepath)
    df_main['date'] =  pd.to_datetime(df_main['date'], format='%Y-%m-%d')
    df_main = df_main.sort_values(['date', 'tau'], ascending = [True, True])

    # Add maturity groups and bias corrected bubble estimate
    df_main['tau_years'] = df_main['tau']/365 # tau in years
    df_main['maturity_group'] = np.where(df_main['tau_years']<8/365, "vlow",  np.where((df_main['tau_years']<0.25)&(df_main['tau_years']<0.25), "low", np.where(df_main['tau_years']>0.5, "high", "med"))) #

    # Impute nonsensible values - replace with previous value if se > 1000
    semax = 1000
    if name == 'GOOGL':
        semax = 500
    df_main['sbub_qcdf_se'] = np.real(df_main['sbub_qcdf_se'])
    s = df_main['sbub_qcdf_se'] > semax # Find where se >1000
    df_main.loc[s, "sbub_qcdf_se"] = np.nan # Set these equal to NAN
    df_main["sbub_qcdf_se"].ffill(inplace=True) # Fill NAN with previous valid obs
    
    df_main['sbub_qcdfp_se'] = np.real(df_main['sbub_qcdf_se'])
    s = df_main['sbub_qcdfp_se'] > semax # Find where se >1000
    df_main.loc[s, "sbub_qcdfp_se"] = np.nan # Set these equal to NAN
    df_main["sbub_qcdfp_se"].ffill(inplace=True) # Fill NAN with previous valid obs
    
    df_main['sbub_qcdfc_se'] = np.real(df_main['sbub_qcdf_se'])
    s = df_main['sbub_qcdfc_se'] > semax # Find where se >1000
    df_main.loc[s, "sbub_qcdfc_se"] = np.nan # Set these equal to NAN
    df_main["sbub_qcdfc_se"].ffill(inplace=True) # Fill NAN with previous valid obs
    
    df_main=df_main[df_main['qcdf_bias']<1000]
    df_main=df_main[df_main['qcdfp_bias']<1000]
    df_main=df_main[df_main['qcdfc_bias']<1000]
    df_main=df_main[df_main['qcdfc_bias']>-1000]
    df_main=df_main[df_main['qcdfp_bias']>-1000]
    df_main=df_main[df_main['qcdfc_bias']>-1000]

    # Setting up loop and variables
    tau_grps = ['low', 'med', 'high', 'all']

    period = df_main.date.unique()
    nperiods = len(df_main.date.unique())
    ntaugrps = len(tau_grps)
    nem = 3 #number of estimation methods
    mc = np.ceil(ma**0.25);

    # for collecting results
    bub_gp = np.zeros((nperiods, ntaugrps, nem))
    sbubcdf_mu = np.zeros((nperiods, ntaugrps, nem)) ### Den her giver ikke mening, bare en tom en som de sætter ind i NW_COV, giver 0???
    sbubcdf_se = np.zeros((nperiods, ntaugrps, nem))
    sbubcdf_bc_mu = np.zeros((nperiods, ntaugrps, nem))
    sbubcdf_bc_lb = np.zeros((nperiods, ntaugrps, nem))
    sbubcdf_bc_ub = np.zeros((nperiods, ntaugrps, nem))
    sbubcdf_bias = np.zeros((nperiods, ntaugrps, nem))

    # Estimates by day and group
    for t, date in enumerate(period):
        for j, grp in enumerate(tau_grps):

            # Subset date
            df = df_main[df_main['date'] == date] # date
        
            # Select group
            if grp in ['low', 'med', 'high']: # if group is low, med or high, then select group otherwise include all
                # Subset grp
                df = df[df['maturity_group'] == grp] # group

            # Bias corrected estimates
            bub_gp[t, j, 0] = np.nanmean(df['sbub_qcdfp'] + df['qcdfp_bias']);
            bub_gp[t, j, 1] = np.nanmean(df['sbub_qcdfc'] + df['qcdfc_bias']);
            bub_gp[t, j, 2] = np.nanmean(df['sbub_qcdf'] + df['qcdf_bias']);
            sbubcdf_bc_mu[t, j, 0] = bub_gp[t, j, 0] 
            sbubcdf_bc_mu[t, j, 1] = bub_gp[t, j, 1] 
            sbubcdf_bc_mu[t, j, 2] = bub_gp[t, j, 2] 

            # Standard error 
            ngp = len(df)
            sbubcdf_se[t, j, 0] = np.sqrt(np.nansum(df['sbub_qcdfp_se']**2))/ngp;  
            sbubcdf_se[t, j, 1] = np.sqrt(np.nansum(df['sbub_qcdfc_se']**2))/ngp;  
            sbubcdf_se[t, j, 2] = np.sqrt(np.nansum(df['sbub_qcdf_se']**2))/ngp;  

            # Bounds of A - used for confidence interval
            sbubcdf_bc_lb[t, j, 0] = bub_gp[t, j, 0] - np.nanmean(df['qcdf_Ap_ub']);
            sbubcdf_bc_ub[t, j, 0] = bub_gp[t, j, 0] - np.nanmean(df['qcdf_Ap_lb']);
            sbubcdf_bc_lb[t, j, 1] = bub_gp[t, j, 1] - np.nanmean(df['qcdf_Ac_ub']);
            sbubcdf_bc_ub[t, j, 1] = bub_gp[t, j, 1] - np.nanmean(df['qcdf_Ac_lb']);
            sbubcdf_bc_lb[t, j, 2] = bub_gp[t, j, 2] - np.nanmean(df['qcdf_A_ub']);
            sbubcdf_bc_ub[t, j, 2] = bub_gp[t, j, 2] - np.nanmean(df['qcdf_A_lb']);

            #Bias
            sbubcdf_bias[t, j, 0] = np.nanmean(df['qcdfp_bias']);
            sbubcdf_bias[t, j, 1] = np.nanmean(df['qcdfc_bias']);
            sbubcdf_bias[t, j, 2] = np.nanmean(df['qcdf_bias']); 


    # Used for plot the moving averages - collecting results
    rsbubcdf_mu = np.zeros((nperiods, ntaugrps, nem))
    rsbubcdf_se = np.zeros((nperiods, ntaugrps, nem))
    rsbubcdf_bc_mu = np.zeros((nperiods, ntaugrps, nem))
    rsbubcdf_bc_lb = np.zeros((nperiods, ntaugrps, nem))
    rsbubcdf_bc_ub = np.zeros((nperiods, ntaugrps, nem))

    # Moving averages:
    for t in np.arange(ma, nperiods):
        vt = np.arange(t-ma+1, t) # window of observations
        for j, grp in enumerate(tau_grps):
            for i in np.arange(0, nem): # estimation methods
                #print(t, vt, j)

                # Bubble moving average estimate
                rsbubcdf_bc_mu[t,j,i] = np.nanmean(sbubcdf_bc_mu[vt,j,i]);

                # Newey-West Standard error
                rsbubcdf_mu[t,j,i] = np.nanmean(sbubcdf_mu[vt,j,i])
                rsbubcdf_se[t,j,i] = np.sqrt((np.nansum(sbubcdf_se[vt,j,i]**2)))/ma

                # Confidence interval
                rsbubcdf_bc_lb[t,j,i] = np.nanmean(sbubcdf_bc_lb[vt,j,i]) - 1.96*rsbubcdf_se[t,j,i]
                rsbubcdf_bc_ub[t,j,i] = np.nanmean(sbubcdf_bc_ub[vt,j,i]) + 1.96*rsbubcdf_se[t,j,i]


    # Formatted results - bubble point estimates to use in trading strategies
    # Formatted results - bubble point estimates to use in trading strategies
    date = pd.DataFrame(period)
    bubplow = pd.DataFrame(bub_gp[:,0,0])
    bubpmed = pd.DataFrame(bub_gp[:,1,0])
    bubphigh = pd.DataFrame(bub_gp[:,2,0])
    bubpall = pd.DataFrame(bub_gp[:,3,0])

    bubclow = pd.DataFrame(bub_gp[:,0,1])
    bubcmed = pd.DataFrame(bub_gp[:,1,1])
    bubchigh = pd.DataFrame(bub_gp[:,2,1])
    bubcall = pd.DataFrame(bub_gp[:,3,1])

    bubcplow = pd.DataFrame(bub_gp[:,0,2])
    bubcpmed = pd.DataFrame(bub_gp[:,1,2])
    bubcphigh = pd.DataFrame(bub_gp[:,2,2])
    bubcpall = pd.DataFrame(bub_gp[:,3,2])

    frames = [date, bubcplow, bubplow, bubclow, bubcpmed, bubpmed, bubcmed, bubcphigh, bubphigh, bubchigh, bubpall, bubcall, bubcpall]
    variables = ['date', 'bubcplow', 'bubplow', 'bubclow', 'bubcpmed', 'bubpmed', 'bubcmed', 'bubcphigh', 'bubphigh', 'bubchigh', 'bubpall', 'bubcall', 'bubcpall']

    # Add stock price - #YF
    # Add correct stock price
    sout = pd.read_csv (Path(f'{pf}/data/input/stockprices_{name}.csv')) # Data
    sout[name] = sout['snp']
    sout = sout[sout.date <= end_dates[-1]]
    sout['sout'] = sout['snp']
    sout = sout[['date', 'sout']]
    sout['date'] =  pd.to_datetime(sout['date'], format='%Y-%m-%d')

    # Add treasury rate
    df_trate_1mo3mo = pd.read_excel(f'{pf}/data/input/trate_1mo3mo.xlsx', index_col=0).reset_index() 
    df_tr = df_trate_1mo3mo[['date', 'tr']]
    df_tr = df_tr.drop_duplicates()
    df_tr['date'] =  pd.to_datetime(df_tr['date'], format='%Y-%m-%d')

    df = pd.concat(frames, axis=1)
    df.columns=variables
    df['date'] =  pd.to_datetime(df['date'], format='%Y-%m-%d')
    df = pd.merge(df, sout, how='inner', on='date') #  merge with sout
    df = pd.merge(df, df_tr, how='inner', on='date') # merge with tr
    df_main = pd.merge(df_main, df_tr, how='inner', on='date') # merge main with tr
    df['year'] = df.date.dt.year

    print(filepath)
    return(df)


################################################################################################




def get_combined_bub_estimates(name, volume_weighted, ma, start_dates, end_dates):
        
    # get bub estimates    
    df_bub=get_bub_estimates(name, volume_weighted, False, ma, start_dates, end_dates)
        
    # Fill NAN with previous valid obs estimates
    df_bub = df_bub.fillna(method='ffill')
    
    # Create bub_gp
    period = df_bub.date.unique()
    nperiods = len(df_bub.date.unique())
    ntaugrps = 4 # 4 groups
    nem = 3
    bub_gp = np.zeros((nperiods, ntaugrps, nem))
    bub_gp[:,0,0] = df_bub['bubplow']
    bub_gp[:,1,0] = df_bub['bubpmed']
    bub_gp[:,2,0] = df_bub['bubphigh']
    bub_gp[:,3,0] = df_bub['bubpall']
    bub_gp[:,0,1] = df_bub['bubclow']
    bub_gp[:,1,1] = df_bub['bubcmed']
    bub_gp[:,2,1] = df_bub['bubchigh']
    bub_gp[:,3,1] = df_bub['bubcall']
    bub_gp[:,0,2] = df_bub['bubcplow']
    bub_gp[:,1,2] = df_bub['bubcpmed']
    bub_gp[:,2,2] = df_bub['bubcphigh']
    bub_gp[:,3,2] = df_bub['bubcpall']    
    
    return(df_bub, bub_gp)
    

## Get all bub estimates

In [ ]:
ma = 63

start_dates = ['1996-01-01','1997-01-01','1998-01-01','1999-01-01','2000-01-01','2001-01-01','2002-01-01','2003-01-01','2004-01-01','2005-01-01','2006-01-01','2007-01-01','2008-01-01','2009-01-01','2010-01-01','2011-01-01','2012-01-01', '2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01',
          '2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01']
end_dates = ['1996-12-31','1997-12-31','1998-12-31','1999-12-31','2000-12-31','2001-12-31','2002-12-31','2003-12-31','2004-12-31','2005-12-31','2006-12-31','2007-12-31','2008-12-31','2009-12-31','2010-12-31','2011-12-31','2012-12-31', '2013-12-31', '2014-12-31', '2015-12-31', '2016-12-31', '2017-12-31',
          '2018-12-31', '2019-12-31', '2020-12-31', '2021-12-31']

weights = [False, True, 'Naive'] #Volume weighted = True / False, if false then jarrow weights

# Sp500 bubbles
df_bub_false, bub_gp_false= get_combined_bub_estimates('SPX',False, ma, start_dates, end_dates)
df_bub_true, bub_gp_true= get_combined_bub_estimates('SPX', True, ma, start_dates, end_dates)
df_bub_naive, bub_gp_naive= get_combined_bub_estimates('SPX', 'Naive', ma, start_dates, end_dates)

# NDX bubbles
df_bub_false_ndx, bub_gp_false_ndx = get_combined_bub_estimates('NDX', False, ma, start_dates, end_dates)
df_bub_true_ndx, bub_gp_true_ndx = get_combined_bub_estimates('NDX', True, ma, start_dates, end_dates)

# Trading

## Trade function

In [ ]:
# Med alle 
def Trading_strat_2(df, bub_gp, hdwin, tranr, ma1, ma2, start, end, strat, stats):
    'Trading strategies for a given trading freq (hdwin)'
    'nem=0: only put, nem=1: only call, nem=2: call and put'
    'tranr = transaction cost'

    df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')

    # Find index for bub start when other periods
    df2 = df.drop_duplicates(subset=['date']).reset_index()
    df2 = df2[df2['date'].between(start, end)]
    df2 = pd.DataFrame(df2['date'])
    
    bubstart = df2.index[0]
    bubend = df2.index[-1]
    
    #Slice bubgp
    bub_gp[bubstart:bubend]
    
    # Slice df
    df = df[df['date'].between(start, end)]
    
    start_year = (min(df["date"])).year
    end_year = (max(df["date"])).year
    yr = range(start_year+1,end_year+1)
    nyr = len(yr)


    # Trading parameters
    #tranr = tranr                           # transaction cost
    #ind_ts = 1                              # momentum trading strategy indicator ind_ts=1 for S or Pi; =2 for S and Pi; =3 for Pi only 
    #dwin = 10                               # step for returns in correl analysis
    nem = 3                                  # Methods call, put and both

    # endpoints for tau groups
    taucut = np.array((0,0.25,0.5,1, 1))        
    
    ntaugp = len(taucut)-1 
    #vlag = np.array((range(1,6)))

    # Moving average length
    ma1 = ma1
    ma2 = ma2

    # Creating stock price
    df = df.drop_duplicates(subset=['date']).reset_index()
    snp = df['sout']

    # Periods
    period = df['date']
    nperiod = len(period)

    # T-rate for excess return
    trate = df["tr"]

    # Creating empty variables
    rbubmu_gp = np.zeros((nperiod,ntaugp, nem)); r2bubmu_gp = np.zeros((nperiod, ntaugp, nem)); buy_bub = np.zeros((nperiod, ntaugp, nem))
    buy_rbub = np.zeros((nperiod)); retur_mkt = np.zeros((nperiod))
    rsout = np.zeros((nperiod)); r2sout = np.zeros((nperiod))
    
    pbuy_bub = np.zeros((ntaugp,nem)); sumprofi_bub = np.zeros((ntaugp,nem)); sumnetprofi_bub = np.zeros((ntaugp,nem));
    nbuy_bub = np.zeros((ntaugp,nem)); nsell_bub = np.zeros((ntaugp,nem));
    sumpprofi_bub = np.zeros((ntaugp,nem)); sumnprofi_bub = np.zeros((ntaugp,nem));
    yrretur_bub = np.zeros((ntaugp,nem)); yrnetr_bub = np.zeros((ntaugp,nem));
    yrnetse_bub = np.zeros((ntaugp,nem)); yrnetexr_bub = np.zeros((ntaugp,nem));

    # Stock and bubble for the first moving average period
    for t in range((ma1-1),nperiod):
        vt = np.arange(t-ma1+1,t+1)
        rsout[t] = np.nanmean(snp[vt])
#        rbubmu_gp[t,:,:] = np.nanmean(bub_gp[vt,:,:]);
        for i in range(0, ntaugp):
            for k in range(0,nem):
                rbubmu_gp[t,i,k] = np.nanmean(bub_gp[vt,i,k])

    # Stock and bubble for the second moving average period
    for t in range(ma2,nperiod):
        r2sout[t] = np.nanmean(snp[np.arange(np.maximum(0,t-ma2),t+1)])
#        r2bubmu_gp[t,:,:] = np.nanmean(bub_gp[np.maximum(0,t-ma2):t,:,:])
#        r2bubmu_gp(t,:,:) = nanmean(bub_gp(max(1,t-ma2):t,:,:)); 
        
        for i in range(0, ntaugp):
            for k in range(0,nem):
                r2bubmu_gp[t,i,k] = np.nanmean(bub_gp[np.arange(np.maximum(0,t-ma2),t+1),i,k])  

    # Trading window
    vtr =  np.arange(ma2,nperiod-hdwin,hdwin)
    nvtr = len(vtr)

    #Creating containers for bubble results
    profi_bub = np.zeros((nperiod-hdwin,ntaugp,nem))
    netprofi_bub = np.zeros((nperiod-hdwin,ntaugp,nem))
    retur_bub = np.zeros((nperiod-hdwin,ntaugp,nem))

    
    # Pure momentum and market
    profi_rbub = np.zeros((nperiod-hdwin))
    netprofi_rbub = np.zeros((nperiod-hdwin));
    retur_rbub = np.zeros((nperiod-hdwin));
    retur_mkt = np.zeros((nperiod-hdwin));  
    

    # Trading
    for t in vtr:
        # Pure momentum strategy 
        # Checking if MA1 is higher tha MA2 
        buy_rbub[t] = int(rsout[t] > r2sout[t]);
        
        # Storing result
        profi_rbub[t] = buy_rbub[t]*(snp[t+hdwin]-snp[t]);
        netprofi_rbub[t] = profi_rbub[t]*(1-tranr);
        retur_rbub[t] = buy_rbub[t]*(np.log(snp[t+hdwin])-np.log(snp[t]));       
        retur_mkt[t] = np.log(snp[t+hdwin])-np.log(snp[t]);
                    
        for i in range(0,ntaugp):
            for k in range(0,nem):
                if strat==1:
                    # Checking if bub or stock ma1 is higher than ma2
                    buy_bub[t,i,k] = max(int(rbubmu_gp[t,i,k] > r2bubmu_gp[t,i,k]),int(rsout[t]>r2sout[t]));
                if strat==2:
                    # Only bub
                    buy_bub[t,i,k] = int(rbubmu_gp[t,i,k] > r2bubmu_gp[t,i,k])
                if strat==3:
                    # bub and mom strat
                    buy_bub[t,i,k] =int(rbubmu_gp[t,i,k] > r2bubmu_gp[t,i,k])*int(rsout[t]>r2sout[t])
                if strat==4:
                    # bub all and sout
                    buy_bub[t,i,k] = max(int(rbubmu_gp[t,i,0] > r2bubmu_gp[t,i,0]),int(rbubmu_gp[t,i,1] > r2bubmu_gp[t,i,1]),
                                     int(rbubmu_gp[t,i,2] > r2bubmu_gp[t,i,2]),int(rsout[t]>r2sout[t]));
                if strat==5:
                    buy_bub[t,i,k] = max(int(rbubmu_gp[t,0,k] > r2bubmu_gp[t,0,k]),int(rbubmu_gp[t,1,k] > r2bubmu_gp[t,1,k]),
                                     int(rbubmu_gp[t,2,k] > r2bubmu_gp[t,2,k]), int(rsout[t]>r2sout[t]));


                # Storing result
                profi_bub[t,i,k] = buy_bub[t,i,k]*(snp[t+hdwin]-snp[t]);
                netprofi_bub[t,i,k] = profi_bub[t,i,k]*(1-tranr);
                retur_bub[t,i,k] = buy_bub[t,i,k]*(np.log(snp[t+hdwin])-np.log(snp[t]));   

    # For net return bub strategy and rbub is for momentum
    netr_bub = retur_bub.copy()
    netr_rbub = retur_rbub.copy()

    # MA2 + frequency window
    vtr1 =  np.arange(hdwin + ma2,nperiod-hdwin,hdwin)
    nvtr1 = len(vtr1);

    tranc_bub = np.zeros((nperiod-hdwin,ntaugp,nem));
    tranc_rbub = np.zeros((nperiod-hdwin))

    # Transaction costs
    for t in vtr1:
        for k in range(0, nem):  
            # For bubble strategy
            tranc_bub[t,:,k] = buy_bub[t,:,k] - buy_bub[t-hdwin,:,k]                      
            netr_bub[t,:,k] = netr_bub[t,:,k]   - abs(tranc_bub[t,:,k])*tranr;  

        # Momentum
        tranc_rbub[t] = buy_rbub[t]-buy_rbub[t-hdwin];
        netr_rbub[t] = netr_rbub[t]-np.abs(tranc_rbub[t])*tranr; 


    for k in range(0,nem):
        pbuy_bub[:,k] = np.transpose(np.mean(buy_bub[vtr,:,k]));
        sumprofi_bub[:,k] = np.transpose(np.sum(profi_bub[vtr,:,k]));
        sumnetprofi_bub[:,k] = np.transpose(np.sum(netprofi_bub[vtr,:,k]));

    pbuy_rbub = np.transpose(np.mean(buy_rbub[vtr]));
    sumprofi_rbub = np.transpose(np.sum(profi_rbub[vtr]));
    sumnetprofi_rbub = np.transpose(np.sum(netprofi_rbub[vtr]));

    # Bubble strategy stats
    for i in range (0, ntaugp):   
        for k in range(0,nem):
            nbuy_bub[i,k] = np.sum(tranc_bub[vtr,i,k]>0);
            nsell_bub[i,k] = np.sum(tranc_bub[vtr,i,k]<0);
            sumpprofi_bub[i,k] = np.sum(np.maximum(profi_bub[vtr,i,k],0));
            sumnprofi_bub[i,k] = -np.sum(np.minimum(profi_bub[vtr,i,k],0));
            # Yearly returns
            yrretur_bub[i,k] = np.mean(retur_bub[vtr,i,k])*252/hdwin;
            yrnetr_bub[i,k] = np.mean(netr_bub[vtr,i,k])*252/hdwin;
            yrnetse_bub[i,k] = np.std(netr_bub[vtr,i,k])*np.sqrt(252/hdwin);            
            yrnetexr_bub[i,k] = np.mean(netr_bub[vtr,i,k]*252/hdwin - trate[vtr]);       #mean excess return

    # Momentum stats
    nbuy_rbub = sum(tranc_rbub[vtr1]>0)
    nsell_rbub = sum(tranc_rbub[vtr1]<0);        
    sumpprofi_rbub = sum(np.maximum(profi_rbub[vtr],0));
    sumnprofi_rbub = -sum(np.minimum(profi_rbub[vtr],0)); 

    # Yearly returns
    yrretur_rbub = np.mean(retur_rbub[vtr])*252/hdwin;
    yrnetr_rbub = np.mean(netr_rbub[vtr])*252/hdwin;  
    yrnetse_rbub = np.std(netr_rbub[vtr])*np.sqrt(252/hdwin);

    #mean excess return
    yrnetexr_rbub = np.mean(netr_rbub[vtr]*252/hdwin - trate[vtr]);  # fiks det
    yrnetexr_mkt = np.mean(retur_mkt[vtr]*252/hdwin - trate[vtr]);
    yrreturse_mkt = np.std(retur_mkt[vtr])*np.sqrt(252/hdwin);
    yrretur_mkt = (np.log(snp[len(snp)-1])-np.log(snp[ma2]))/nyr;

    # Number of trans
    ntranc_rbub = nbuy_rbub + nsell_rbub
    ntranc_bub = nbuy_bub + nsell_bub;

    # Sharpe ratio
    yrnetsrpraw_bub = yrnetr_bub / yrnetse_bub;
    yrnetsrp_bub = yrnetexr_bub / yrnetse_bub;
    yrnetsrpraw_rbub = yrnetr_rbub/yrnetse_rbub;
    yrnetsrp_rbub = yrnetexr_rbub/yrnetse_rbub;
    yrnetsrp_mkt = yrnetexr_mkt/yrreturse_mkt;
    
    
    # Collect results in table
    variables = ['Gross', 'Net' , 'SR']
    
    if stats == 'transactions': #return count of transactions instead of gross
        variables = ['Trades', 'Net' , 'SR']
    if stats == 'exposure': #return exposure (time in the market) instead of gross
        variables = ['Exp', 'Net' , 'SR']
    if stats == 'both': # return exposure, transaction count and sharpe
        variables = ['Exp', 'Trades' , 'SR']
    
    strategies_str = [r'$\hat{\Pi}_{cp}(\tau_{low})$', r'$\hat{\Pi}_{c}(\tau_{low})$', r'$\hat{\Pi}_{p}(\tau_{low})$',
                      r'$\hat{\Pi}_{cp}(\tau_{med})$', r'$\hat{\Pi}_{c}(\tau_{med})$', r'$\hat{\Pi}_{p}(\tau_{med})$',
                      r'$\hat{\Pi}_{cp}(\tau_{high})$', r'$\hat{\Pi}_{c}(\tau_{high})$', r'$\hat{\Pi}_{p}(\tau_{high})$',
                      r'$\hat{\Pi}_{cp}(\tau_{all})$', r'$\hat{\Pi}_{c}(\tau_{all})$', r'$\hat{\Pi}_{p}(\tau_{all})$',
                      'PM',r'Buy\&hold']
    
    results = np.zeros((14,3)); results = pd.DataFrame(results); results.columns = variables; results = results.set_index([strategies_str])
            
    for i, k in zip(range(0,ntaugp), reversed(range(0,nem))):
        # low
        results.iloc[i,0] = round(yrretur_bub[0,k]*100,1)
        results.iloc[i,1] = round(yrnetr_bub[0,k]*100,1)
        results.iloc[i,2] = round(yrnetsrp_bub[0,k],3)
        
        # med
        results.iloc[i+3,0] = round(yrretur_bub[1,k]*100,1)
        results.iloc[i+3,1] = round(yrnetr_bub[1,k]*100,1)
        results.iloc[i+3,2] = round(yrnetsrp_bub[1,k],3)
        
        # high
        results.iloc[i+6,0] = round(yrretur_bub[2,k]*100,1)
        results.iloc[i+6,1] = round(yrnetr_bub[2,k]*100,1)
        results.iloc[i+6,2] = round(yrnetsrp_bub[2,k],3)
    
        # all
        results.iloc[i+9,0] = round(yrretur_bub[3,k]*100,1)
        results.iloc[i+9,1] = round(yrnetr_bub[3,k]*100,1)
        results.iloc[i+9,2] = round(yrnetsrp_bub[3,k],3)
        
        if stats == 'transactions': #return count of transactions instead of gross
            results.iloc[i,0] = round(ntranc_bub[0,k],0)
            results.iloc[i+3,0] = round(ntranc_bub[1,k],0)
            results.iloc[i+6,0] = round(ntranc_bub[2,k],0)
            results.iloc[i+9,0] = round(ntranc_bub[3,k],0)
            
        if stats == 'exposure': #return count of transactions instead of gross
            results.iloc[i,0] = round(np.mean(buy_bub[:,0,k]*hdwin)*100,1)
            results.iloc[i+3,0] = round(np.mean(buy_bub[:,1,k]*hdwin)*100,1)
            results.iloc[i+6,0] = round(np.mean(buy_bub[:,2,k]*hdwin)*100,1)
            results.iloc[i+9,0] = round(np.mean(buy_bub[:,3,k]*hdwin)*100,1)
            
        if stats == 'both':
            # Exposure
            results.iloc[i,0] = round(np.mean(buy_bub[:,0,k]*hdwin)*100,1)
            results.iloc[i+3,0] = round(np.mean(buy_bub[:,1,k]*hdwin)*100,1)
            results.iloc[i+6,0] = round(np.mean(buy_bub[:,2,k]*hdwin)*100,1)
            results.iloc[i+9,0] = round(np.mean(buy_bub[:,3,k]*hdwin)*100,1)
            
            # Transactions
            results.iloc[i,1] = round(ntranc_bub[0,k],0)
            results.iloc[i+3,1] = round(ntranc_bub[1,k],0)
            results.iloc[i+6,1] = round(ntranc_bub[2,k],0)
            results.iloc[i+9,1] = round(ntranc_bub[3,k],0)


    # Momentum
    results.iloc[12,0] = round(yrretur_rbub*100,1)
    results.iloc[12,1] = round(yrnetr_rbub*100,1)
    results.iloc[12,2] = round(yrnetsrp_rbub,3)
    
    #Market
    results.iloc[13,0:2] = round(yrretur_mkt*100,1)
    results.iloc[13,2] = round(yrnetsrp_mkt,3)
    
    if stats == 'transactions': #return count of transactions instead of gross return
        results.iloc[12,0] = round(ntranc_rbub,0)
        results.iloc[13,0] = 1
        
    if stats == 'exposure': #return count of transactions instead of gross
        results.iloc[12,0] = round(np.mean(buy_rbub)*hdwin*100,1)
        results.iloc[13,0] = 100
        
    if stats == 'both':
        # Exposure
        results.iloc[12,0] = round(np.mean(buy_rbub)*hdwin*100,1)
        results.iloc[13,0] = 100
        
        # Transactions
        results.iloc[12,1] = round(ntranc_rbub,0)
        results.iloc[13,1] = 1
        

    return results, yrretur_bub, yrnetr_bub, ntranc_rbub, yrnetr_rbub, trate, netr_rbub, netr_bub, retur_mkt

## Trading results

In [ ]:
df_bub = df_bub_true
bub_gp = bub_gp_true
strat = 1
stats = 'exposure'

start = '1996-01-01'
end = '2021-12-31'
ma1= 63
ma2 = 252
tranr = 0.005
hdwin = 1
res = Trading_strat_2(df_bub, bub_gp, hdwin, tranr, ma1, ma2, start, end, strat, stats)
#res_ml = Trading_strat(df_bub, bub_gp_ml, hdwin, tranr, ma1, ma2, start, end)
#df = pd.concat([res[0],res_ml[0]], axis=1)
#df['diffsr']=df.iloc[:, 2]-df.iloc[:, 5]
res[0]

In [ ]:
df_p0 = pd.DataFrame(res[6]) # Pure momentum
df_p1 = pd.DataFrame(res[8]) # Buyhold
df_p2 = pd.DataFrame(res[7][:,:,2]) # CP
#df_p5 = pd.DataFrame(res[7][:,:,2])

frames = [df_p0, df_p1, df_p2, df_p3, df_p4]#, df_p5] 
df_results = pd.concat(frames, axis=1)
df_results.to_excel('test.xlsx')

In [ ]:
frames = [df_p0, df_p1, df_p2, df_p3, df_p4]#, df_p5] 
df_results = pd.concat(frames, axis=1)
df_results.to_excel('test.xlsx')

In [ ]:
df_results

### Table: Performance of trading strategies on S\&P 500 over different sample horizons

In [ ]:
df_bub = df_bub_true
bub_gp = bub_gp_true

stats = 'exposure'
start = '1996-01-01'
end = '2021-12-31'
ma1= 63
ma2 = 252
tranr = 0.005
hdwin = 5
strat = 1

# periods to loop over
start_dates = ['1996-01-01', '1996-01-01', '2000-01-01', '2005-01-01', '2010-01-01', '2015-01-01']
end_dates =['2021-12-31', '2000-12-31', '2005-12-31', '2010-12-31', '2015-12-31', '2021-12-31']


strategies = [r'$\hat{\Pi}_{cp}(\tau_{low})$', r'$\hat{\Pi}_{c}(\tau_{low})$', r'$\hat{\Pi}_{p}(\tau_{low})$',
                      r'$\hat{\Pi}_{cp}(\tau_{med})$', r'$\hat{\Pi}_{c}(\tau_{med})$', r'$\hat{\Pi}_{p}(\tau_{med})$',
                      r'$\hat{\Pi}_{cp}(\tau_{high})$', r'$\hat{\Pi}_{c}(\tau_{high})$', r'$\hat{\Pi}_{p}(\tau_{high})$',
                      r'$\hat{\Pi}_{cp}(\tau_{all})$', r'$\hat{\Pi}_{c}(\tau_{all})$', r'$\hat{\Pi}_{p}(\tau_{all})$',
                      'PM',r'Buy\&hold']
variables = ['Gross', 'Net' , 'SR']

if stats == 'transactions': #return count of transactions instead of gross
    variables = ['Trades', 'Net' , 'SR']
if stats == 'exposure': #return count of transactions instead of gross
    variables = ['Exp', 'Net' , 'SR']

results = np.empty((len(start_dates), len(strategies), len(variables))) # number of different parameters, N strategy, N variables

for i, (start, end) in enumerate(zip(start_dates, end_dates)):
    res = Trading_strat_2(df_bub, bub_gp, hdwin, tranr, ma1, ma2, start, end, strat, stats)
    results[i, :, :] = res[0]
    
df_p0 = pd.DataFrame(results[0, :, :])
df_p1 = pd.DataFrame(results[1, :, :])
df_p2 = pd.DataFrame(results[2, :, :])
df_p3 = pd.DataFrame(results[3, :, :])
df_p4 = pd.DataFrame(results[4, :, :])
df_p5 = pd.DataFrame(results[5, :, :])

frames = [df_p0, df_p1, df_p2, df_p3, df_p4, df_p5] 
df_results = pd.concat(frames, axis=1)
df_results.columns=variables*6
df_results = df_results.set_index([strategies])
df_results

# Print to latex
if stats == 'transactions': #return count of transactions instead of gross
    print(df_results.style.format(subset=['Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Trades'], precision=0, escape='Latex').to_latex())
if stats == 'exposure': #return count of transactions instead of gross
    print(df_results.style.format(subset=['Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Exp'], precision=0, escape='Latex').to_latex())
else:
    print(df_results.style.format(subset=['Gross','Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').to_latex())

#### Different periods

In [ ]:
df_bub = df_bub_true
bub_gp = bub_gp_true

stats = 'exposure'
start = '1996-01-01'
end = '2021-12-31'
ma1= 63
ma2 = 252
tranr = 0.005
hdwin = 5
strat = 1

# periods to loop over
start_dates = ['1996-01-01', '1996-01-01', '2015-12-31']
end_dates =['2021-12-31', '2015-12-31', '2021-12-31']


strategies = [r'$\hat{\Pi}_{cp}(\tau_{low})$', r'$\hat{\Pi}_{c}(\tau_{low})$', r'$\hat{\Pi}_{p}(\tau_{low})$',
                      r'$\hat{\Pi}_{cp}(\tau_{med})$', r'$\hat{\Pi}_{c}(\tau_{med})$', r'$\hat{\Pi}_{p}(\tau_{med})$',
                      r'$\hat{\Pi}_{cp}(\tau_{high})$', r'$\hat{\Pi}_{c}(\tau_{high})$', r'$\hat{\Pi}_{p}(\tau_{high})$',
                      r'$\hat{\Pi}_{cp}(\tau_{all})$', r'$\hat{\Pi}_{c}(\tau_{all})$', r'$\hat{\Pi}_{p}(\tau_{all})$',
                      'PM',r'Buy\&hold']
variables = ['Gross', 'Net' , 'SR']

if stats == 'transactions': #return count of transactions instead of gross
    variables = ['Trades', 'Net' , 'SR']
if stats == 'exposure': #return count of transactions instead of gross
    variables = ['Exp', 'Net' , 'SR']

results = np.empty((len(start_dates), len(strategies), len(variables))) # number of different parameters, N strategy, N variables

for i, (start, end) in enumerate(zip(start_dates, end_dates)):
    res = Trading_strat_2(df_bub, bub_gp, hdwin, tranr, ma1, ma2, start, end, strat, stats)
    results[i, :, :] = res[0]
    
df_p0 = pd.DataFrame(results[0, :, :])
df_p1 = pd.DataFrame(results[1, :, :])
df_p2 = pd.DataFrame(results[2, :, :])
#df_p3 = pd.DataFrame(results[3, :, :])
#df_p4 = pd.DataFrame(results[4, :, :])
#df_p5 = pd.DataFrame(results[5, :, :])

frames = [df_p0, df_p1, df_p2]#, df_p3]#, df_p4, df_p5] 
df_results = pd.concat(frames, axis=1)
df_results.columns=variables*len(start_dates) #6
df_results = df_results.set_index([strategies])

df_results = df_results.iloc[[0,3, 6, 9, 12, 13]]
df_results

# Print to latex
if stats == 'transactions': #return count of transactions instead of gross
    print(df_results.style.format(subset=['Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Trades'], precision=0, escape='Latex').to_latex())
if stats == 'exposure': #return count of transactions instead of gross
    print(df_results.style.format(subset=['Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Exp'], precision=0, escape='Latex').to_latex())
else:
    print(df_results.style.format(subset=['Gross','Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').to_latex())


In [ ]:
df_results

### Table: Performance of trading strategies on S&P 500 for different trading frequencies

In [ ]:
df_bub = df_bub_true
bub_gp = bub_gp_true
stats = 'both'

start = '1996-01-01'
end = '2021-12-31'
ma1= 63
ma2 = 252
tranr = 0.005
tradefreq = [1, 5, 10, 20] #hdwin
strat = 1

results = np.empty((len(tradefreq), len(strategies), len(variables))) # number of different parameters, N strategy, N variables

for i, hdwin in enumerate(tradefreq):
    res = Trading_strat_2(df_bub, bub_gp, hdwin, tranr, ma1, ma2, start, end, strat, stats)
    results[i, :, :] = res[0]
    
df_p0 = pd.DataFrame(results[0, :, :])
df_p1 = pd.DataFrame(results[1, :, :])
df_p2 = pd.DataFrame(results[2, :, :])
df_p3 = pd.DataFrame(results[3, :, :])
#df_p4 = pd.DataFrame(results[4, :, :])
#df_p5 = pd.DataFrame(results[5, :, :])


if stats == 'transactions': #return count of transactions instead of gross
    variables = ['Trades', 'Net' , 'SR']
if stats == 'exposure': #return count of transactions instead of gross
    variables = ['Exp', 'Net' , 'SR']
if stats == 'both': #return count of transactions instead of gross
    variables = ['Exp', 'Trades' , 'SR']

frames = [df_p0, df_p1, df_p2, df_p3]#, df_p4]#, df_p5] 
df_results = pd.concat(frames, axis=1)
df_results.columns=variables*4
df_results = df_results.set_index([strategies])
df_results

In [ ]:
# Print to latex
if stats == 'transactions': #return count of transactions instead of gross
    print(df_results.style.format(subset=['Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Trades'], precision=0, escape='Latex').to_latex())
if stats == 'exposure': #return count of transactions instead of gross
    print(df_results.style.format(subset=['Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Exp'], precision=0, escape='Latex').to_latex())
if stats == 'both': #return count of transactions instead of gross
    print(df_results.style.format(subset=['Trades'], precision=0, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Exp'], precision=0, escape='Latex').to_latex())
else:
    print(df_results.style.format(subset=['Gross','Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').to_latex())

### Table: Performance of trading strategies on S&P 500 using different weights

In [ ]:
start = '1996-01-01'
end = '2021-12-31'
stats = 'transactions'
ma1= 63
ma2 = 252
tranr = 0.005
hdwin = 5
strat = 1 #join momentum

res_false = Trading_strat_2(df_bub_false, bub_gp_false, hdwin, tranr, ma1, ma2, start, end, strat, stats)
res_true = Trading_strat_2(df_bub_true, bub_gp_true, hdwin, tranr, ma1, ma2, start, end, strat, stats)
res_naive = Trading_strat_2(df_bub_naive, bub_gp_naive, hdwin, tranr, ma1, ma2, start, end, strat, stats)

In [ ]:
df_p0 = pd.DataFrame(res_naive[0])
df_p1 = pd.DataFrame(res_false[0])
df_p2 = pd.DataFrame(res_true[0])

frames = [df_p0, df_p1, df_p2] 
df_results = pd.concat(frames, axis=1)

# note order: first naive, then false = jarrow weights, then true = volume weighted

df_results.to_excel('table.xlsx')

# Format excel manually

In [ ]:
variables = ['Gross', 'Net' , 'SR']

if stats == 'transactions': #return count of transactions instead of gross
    variables = ['Trades', 'Net' , 'SR']
if stats == 'exposure': #return count of transactions instead of gross
    variables = ['Exp', 'Net' , 'SR']

df_results = pd.read_excel('table_formatted.xlsx')
df_results= df_results.set_index('Index')
df_results.columns=variables*5
# note order: 100% put, then 100% call , naive, jarrow, volume
df_results

In [ ]:
# Print to latex
if stats == 'transactions': #return count of transactions instead of gross
    print(df_results.style.format(subset=['Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Trades'], precision=0, escape='Latex').to_latex())
if stats == 'exposure': #return count of transactions instead of gross
    print(df_results.style.format(subset=['Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Exp'], precision=0, escape='Latex').to_latex())
else:
    print(df_results.style.format(subset=['Gross','Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').to_latex())

### Table: Performance of different strategies

In [ ]:
df_bub = df_bub_false
bub_gp = bub_gp_false

stats = 'exposure'
ma1= 63
ma2 = 252
tranr = 0.005
hdwin = 5
start = "1996-01-01"
end = "2021-12-31"

strat = np.array([1,2,3,4,5])
res_strat = list([0,0,0,0,0])
for i, j in enumerate(strat):
    res= Trading_strat_2(df_bub, bub_gp, hdwin, tranr, ma1, ma2, start, end, j, stats)
    res_strat[i] = res[0]


In [ ]:
Table_strats = pd.DataFrame(pd.concat([res_strat[0], res_strat[1],res_strat[2], res_strat[3], res_strat[4]], axis=1))
#Table_strats.columns = range(len(Table_strats.columns))
Table_strats

In [ ]:
# Print to latex
if stats == 'transactions': #return count of transactions instead of gross
    print(Table_strats.style.format(subset=['Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Trades'], precision=0, escape='Latex').to_latex())
if stats == 'exposure': #return count of transactions instead of gross
    print(Table_strats.style.format(subset=['Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Exp'], precision=0, escape='Latex').to_latex())
else:
    print(Table_strats.style.format(subset=['Gross','Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').to_latex())

### Table: Moving average lengths

In [ ]:
df_bub = df_bub_true
bub_gp = bub_gp_true
stats = 'transactions'

start = '1996-01-01'
end = '2021-12-31'
ma1= 63
ma2 = 252
tranr = 0.005
hdwin = 5
strat = 1

#w1 = [21, 42, 63, 84, 105, 126]
#w2 = [63, 126, 252, 378, 504, 630]

#w1 = [63, 126, 252]
#w2 = [252, 252, 504]

w1 = [21, 42, 63, 126, 252]
w2 = [63, 126, 252, 252, 504]

results = np.empty((len(w1), len(strategies), len(variables))) # number of different parameters, N strategy, N variables

for i, (ma1, ma2) in enumerate(zip(w1, w2)):
    res = Trading_strat_2(df_bub, bub_gp, hdwin, tranr, ma1, ma2, start, end, strat, stats)
    results[i, :, :] = res[0]


In [ ]:
df_p0 = pd.DataFrame(results[0, :, :])
df_p1 = pd.DataFrame(results[1, :, :])
df_p2 = pd.DataFrame(results[2, :, :])
df_p3 = pd.DataFrame(results[3, :, :])
df_p4 = pd.DataFrame(results[4, :, :])
#df_p5 = pd.DataFrame(results[5, :, :])


if stats == 'transactions': #return count of transactions instead of gross
    variables = ['Trades', 'Net' , 'SR']
if stats == 'exposure': #return count of transactions instead of gross
    variables = ['Exp', 'Net' , 'SR']
if stats == 'both': #return count of transactions instead of gross
    variables = ['Exp', 'Trades' , 'SR']

frames = [df_p0, df_p1, df_p2, df_p3, df_p4]#, df_p5] 
df_results = pd.concat(frames, axis=1)
df_results.columns=variables*5
df_results = df_results.set_index([strategies])

#df_results = df_results.iloc[[0,3, 6, 9, 12, 13]]
df_results

In [ ]:

# Print to latex
if stats == 'transactions': #return count of transactions instead of gross
    print(df_results.style.format(subset=['Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Trades'], precision=0, escape='Latex').to_latex())
if stats == 'exposure': #return count of transactions instead of gross
    print(df_results.style.format(subset=['Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Exp'], precision=0, escape='Latex').to_latex())
else:
    print(df_results.style.format(subset=['Gross','Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').to_latex())

### For reality check test

In [ ]:
df_bub = df_bub_true
bub_gp = bub_gp_true
stats = 'transactions'

start = '1996-01-01'
end = '2021-12-31'
ma1= 63
ma2 = 252
tranr = 0.005
hdwin = 5
strat = 1

tradefreq = [1, 5, 10, 20] #hdwin

w1 = [63, 126, 252]
w2 = [252, 252, 504]


####
hdwin = 5
results = np.empty((len(w1), len(strategies), len(variables))) # number of different parameters, N strategy, N variables
puremom = np.empty((6538, len(w1))) # number of different parameters, N strategy, N variables
buyhold = np.empty((6538, len(w1))) # number of different parameters, N strategy, N variables
strats = np.empty((6538, len(w1), 4)) # number of different parameters, N strategy, N variables


for i, (ma1, ma2) in enumerate(zip(w1, w2)):
    res = Trading_strat_2(df_bub, bub_gp, hdwin, tranr, ma1, ma2, start, end, strat, stats)
    
    print(res[6].shape, res[8].shape, res[7][:,:,2].shape)
    results[i, :, :] = res[0]
    puremom[:, i] = res[6]
    buyhold[:, i] = res[8]
    strats[:, i, 0] = res[7][:, 0, 2] #CP, low
    strats[:, i, 1] = res[7][:, 1, 2] #CP, med, 
    strats[:, i, 2] = res[7][:, 2, 2] #CP, high
    strats[:, i, 3] = res[7][:, 3, 2] #CP, all
    
df_p0 = pd.DataFrame(puremom) # Pure momentum
df_p1 = pd.DataFrame(buyhold) # Buyhold
df_p2 = pd.DataFrame(strats[:,:,0]) # CP, low
df_p3 = pd.DataFrame(strats[:,:,1]) # CP, med
df_p4 = pd.DataFrame(strats[:,:,2]) # CP, high
df_p5 = pd.DataFrame(strats[:,:,3]) # CP, all
#df_p5 = pd.DataFrame(res[7][:,:,2])

frames = [df_p0, df_p1, df_p2, df_p3, df_p4, df_p5] 
df_results = pd.concat(frames, axis=1)
df_results.to_excel('wrchdwin5.xlsx')

In [ ]:
df_p0

In [ ]:
####
hdwin = 1
results = np.empty((len(w1), len(strategies), len(variables))) # number of different parameters, N strategy, N variables
puremom = np.empty((6542, len(w1))) # number of different parameters, N strategy, N variables
buyhold = np.empty((6542, len(w1))) # number of different parameters, N strategy, N variables
strats = np.empty((6542, len(w1), 4)) # number of different parameters, N strategy, N variables


for i, (ma1, ma2) in enumerate(zip(w1, w2)):
    res = Trading_strat_2(df_bub, bub_gp, hdwin, tranr, ma1, ma2, start, end, strat, stats)
    
    print(res[6].shape, res[8].shape, res[7][:,:,2].shape)
    results[i, :, :] = res[0]
    puremom[:, i] = res[6]
    buyhold[:, i] = res[8]
    strats[:, i, 0] = res[7][:, 0, 2] #CP, low
    strats[:, i, 1] = res[7][:, 1, 2] #CP, med, 
    strats[:, i, 2] = res[7][:, 2, 2] #CP, high
    strats[:, i, 3] = res[7][:, 3, 2] #CP, all
    
df_p0 = pd.DataFrame(puremom) # Pure momentum
df_p1 = pd.DataFrame(buyhold) # Buyhold
df_p2 = pd.DataFrame(strats[:,:,0]) # CP, low
df_p3 = pd.DataFrame(strats[:,:,1]) # CP, med
df_p4 = pd.DataFrame(strats[:,:,2]) # CP, high
df_p5 = pd.DataFrame(strats[:,:,3]) # CP, all
#df_p5 = pd.DataFrame(res[7][:,:,2])

frames = [df_p0, df_p1, df_p2, df_p3, df_p4, df_p5] 
df_results = pd.concat(frames, axis=1)
df_results.to_excel('wrchdwin1.xlsx')

In [ ]:
####
hdwin = 10
results = np.empty((len(w1), len(strategies), len(variables))) # number of different parameters, N strategy, N variables
puremom = np.empty((6533, len(w1))) # number of different parameters, N strategy, N variables
buyhold = np.empty((6533, len(w1))) # number of different parameters, N strategy, N variables
strats = np.empty((6533, len(w1), 4)) # number of different parameters, N strategy, N variables


for i, (ma1, ma2) in enumerate(zip(w1, w2)):
    res = Trading_strat_2(df_bub, bub_gp, hdwin, tranr, ma1, ma2, start, end, strat, stats)
    
    print(res[6].shape, res[8].shape, res[7][:,:,2].shape)
    results[i, :, :] = res[0]
    puremom[:, i] = res[6]
    buyhold[:, i] = res[8]
    strats[:, i, 0] = res[7][:, 0, 2] #CP, low
    strats[:, i, 1] = res[7][:, 1, 2] #CP, med, 
    strats[:, i, 2] = res[7][:, 2, 2] #CP, high
    strats[:, i, 3] = res[7][:, 3, 2] #CP, all
    
df_p0 = pd.DataFrame(puremom) # Pure momentum
df_p1 = pd.DataFrame(buyhold) # Buyhold
df_p2 = pd.DataFrame(strats[:,:,0]) # CP, low
df_p3 = pd.DataFrame(strats[:,:,1]) # CP, med
df_p4 = pd.DataFrame(strats[:,:,2]) # CP, high
df_p5 = pd.DataFrame(strats[:,:,3]) # CP, all
#df_p5 = pd.DataFrame(res[7][:,:,2])

frames = [df_p0, df_p1, df_p2, df_p3, df_p4, df_p5] 
df_results = pd.concat(frames, axis=1)
df_results.to_excel('wrchdwin10.xlsx')

In [ ]:
####
hdwin = 20
results = np.empty((len(w1), len(strategies), len(variables))) # number of different parameters, N strategy, N variables
puremom = np.empty((6523, len(w1))) # number of different parameters, N strategy, N variables
buyhold = np.empty((6523, len(w1))) # number of different parameters, N strategy, N variables
strats = np.empty((6523, len(w1), 4)) # number of different parameters, N strategy, N variables


for i, (ma1, ma2) in enumerate(zip(w1, w2)):
    res = Trading_strat_2(df_bub, bub_gp, hdwin, tranr, ma1, ma2, start, end, strat, stats)
    
    print(res[6].shape, res[8].shape, res[7][:,:,2].shape)
    results[i, :, :] = res[0]
    puremom[:, i] = res[6]
    buyhold[:, i] = res[8]
    strats[:, i, 0] = res[7][:, 0, 2] #CP, low
    strats[:, i, 1] = res[7][:, 1, 2] #CP, med, 
    strats[:, i, 2] = res[7][:, 2, 2] #CP, high
    strats[:, i, 3] = res[7][:, 3, 2] #CP, all
    
df_p0 = pd.DataFrame(puremom) # Pure momentum
df_p1 = pd.DataFrame(buyhold) # Buyhold
df_p2 = pd.DataFrame(strats[:,:,0]) # CP, low
df_p3 = pd.DataFrame(strats[:,:,1]) # CP, med
df_p4 = pd.DataFrame(strats[:,:,2]) # CP, high
df_p5 = pd.DataFrame(strats[:,:,3]) # CP, all
#df_p5 = pd.DataFrame(res[7][:,:,2])

frames = [df_p0, df_p1, df_p2, df_p3, df_p4, df_p5] 
df_results = pd.concat(frames, axis=1)
df_results.to_excel('wrchdwin20.xlsx')

### Nasdaq

In [ ]:
df_bub = df_bub_true_ndx
bub_gp = bub_gp_true_ndx
strat = 1
stats = 'exposure'

start = '1996-01-01'
end = '2021-12-31'
ma1= 63
ma2 = 252
tranr = 0.005
hdwin = 5
res = Trading_strat_2(df_bub, bub_gp, hdwin, tranr, ma1, ma2, start, end, strat, stats)
#res_ml = Trading_strat(df_bub, bub_gp_ml, hdwin, tranr, ma1, ma2, start, end)
#df = pd.concat([res[0],res_ml[0]], axis=1)
#df['diffsr']=df.iloc[:, 2]-df.iloc[:, 5]
res[0]

# Plots

In [ ]:
#strat = 1
#buy_bub = begge
#buy_rbub = kun sp500


df_bub = df_bub_false
bub_gp = bub_gp_false
strat = 1

start = '1996-01-01'
end = '2021-12-31'
ma1= 63
ma2 = 252
tranr = 0.005
hdwin = 5
res = Trading_strat_2(df_bub, bub_gp, hdwin, tranr, ma1, ma2, start, end, strat)
#res_ml = Trading_strat(df_bub, bub_gp_ml, hdwin, tranr, ma1, ma2, start, end)
#df = pd.concat([res[0],res_ml[0]], axis=1)
#df['diffsr']=df.iloc[:, 2]-df.iloc[:, 5]
buy_bub = res[-2]
buy_rbub = res[-1]
res[0]

## Bubble estimates moving average

### Different indices

#### Bub estimates

In [ ]:
names = ['SPX','NDX', 'RUT']#, 'TSLA', 'AMZN', 'GOOGL']

ma=63

start_dates = ['1996-01-01','1997-01-01','1998-01-01','1999-01-01','2000-01-01','2001-01-01','2002-01-01','2003-01-01','2004-01-01','2005-01-01','2006-01-01','2007-01-01','2008-01-01','2009-01-01','2010-01-01','2011-01-01','2012-01-01', '2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01',
          '2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01']
end_dates = ['1996-12-31','1997-12-31','1998-12-31','1999-12-31','2000-12-31','2001-12-31','2002-12-31','2003-12-31','2004-12-31','2005-12-31','2006-12-31','2007-12-31','2008-12-31','2009-12-31','2010-12-31','2011-12-31','2012-12-31', '2013-12-31', '2014-12-31', '2015-12-31', '2016-12-31', '2017-12-31',
          '2018-12-31', '2019-12-31', '2020-12-31', '2021-12-31']
start = start_dates[1]
end = end_dates[-1]

volume_weighted = True
spx_bub, spx_gp= get_combined_bub_estimates('SPX',volume_weighted, ma, start_dates, end_dates)
ndx_bub, ndx_gp= get_combined_bub_estimates('NDX',volume_weighted, ma, start_dates, end_dates)
rut_bub, rut_gp= get_combined_bub_estimates('RUT',volume_weighted, ma, start_dates, end_dates)



In [ ]:
col = 'bubcpall'
colors = ['blue', 'red', 'orange', 'green']
labels = ['S&P 500', 'Nasdaq', 'Russel 2000']

start = start_dates[1]
end = end_dates[-1]
print(start, end)

fig, ax = plt.subplots(1, 1, figsize=(10, 6), constrained_layout=True)

# Smoothing insane bubble estimates
spar = 5000
ndx_bubs = ndx_bub[ndx_bub.bubcpall<spar]
ndx_bubs = ndx_bubs[ndx_bub.bubcpall<spar]
rut_bubs = rut_bub[rut_bub.bubcpall<spar]
rut_bubs = rut_bubs[rut_bub.bubcpall>-spar]

# X -Values
x1 = spx_bub['date']
x2 = ndx_bubs['date']
x3 = rut_bubs['date']

y1 = (spx_bub[col]).rolling(ma).mean() #rolling average 3m - 63 trading days
y2 = (ndx_bubs[col]).rolling(ma).mean()
y3 = (rut_bubs[col]).rolling(ma).mean()


# Plot
ax.plot(x1, y1, label = f'{labels[0]}', color=colors[0])
ax.plot(x2, y2, label = f'{labels[1]}', color=colors[1])
ax.plot(x3, y3, label = f'{labels[2]}', color=colors[2])
    

# ax.plot(x1, y1/np.mean(spx_bub['sout']), label = f'{labels[0]}', color=colors[0])
# ax.plot(x2, y2/np.mean(ndx_bubs['sout']), label = f'{labels[1]}', color=colors[1])
# ax.plot(x3, y3/np.mean(rut_bubs['sout']), label = f'{labels[2]}', color=colors[2])
    
# Axes, Labels, title etc 
ax.set_xlim([pd.to_datetime(start, format = '%Y-%m-%d'),pd.to_datetime(end, format = '%Y-%m-%d')])
ax.set_ylabel(r'Bubble estimate')
ax.set_title(r'$\hat{\Pi}_{cp}(\tau)$', loc='center', fontsize='medium')
ax.legend(loc='upper left')
#ax.set_ylim(-60,200)

filepath = Path(f'{pf}/data/figures/bubbles_indices') # Set name
plt.savefig(filepath) #bbox_inches='tight'

In [ ]:
frames = [spx_bub[col], ndx_bubs[col] ,  rut_bubs[col]]
bubs = pd.concat(frames, axis=1)
bubs = bubs.dropna()
bubs.columns = names
bubs.rolling(ma).mean().corr()

#### Level of the indices

In [ ]:
colors = ['blue', 'red', 'orange', 'green']
labels = ['S&P 500', 'Nasdaq', 'Russel 2000']

start = start_dates[0]
end = end_dates[-1]
print(start, end)

fig, ax = plt.subplots(1, 1, figsize=(10, 6), constrained_layout=True)

indices = pd.DataFrame()

for i, name in enumerate(names):
    # Read data
    sout = pd.read_csv (Path(f'{pf}/data/input/stockprices_{name}.csv')) # Data
    sout[name] = sout['snp']
    sout['date'] =  pd.to_datetime(sout['date'], format='%Y-%m-%d')
    sout = sout[sout['date'].between(start, end)]
    
    # X -Values
    x = sout['date']

    #Y - Values 
    y = sout[name]/sout[name].iloc[0]*100
    
    # Plot
    ax.plot(x, y, label = f'{labels[i]}', color=colors[i])
    
    # For calculating cov
    indices[name]=sout[name]

# Axes, Labels, title etc 
ax.set_xlim([pd.to_datetime(start, format = '%Y-%m-%d'),pd.to_datetime(end, format = '%Y-%m-%d')])
ax.set_ylabel(r'Index, 1996-01-02 = 100')
ax.set_title(r'Stock price indices', loc='center', fontsize='medium')
ax.legend(loc='upper left')
    
filepath = Path(f'{pf}/data/figures/descriptive_indices') # Set name
plt.savefig(filepath) #bbox_inches='tight'

corr = indices.corr()
corr
#ax.set_ylim(-10, 100)

#### Trading on different indices

In [ ]:
dfs = [spx_bub, ndx_bub, rut_bub]
bubs = [spx_gp, ndx_gp, rut_gp]

stats = 'exposure'

start = '1996-01-01'
end = '2021-12-31'
ma1= 63
ma2 = 252
tranr = 0.005
hdwin = 5
strat = 1

frames = []

results = np.empty((len(dfs), len(strategies), len(variables))) # number of different parameters, N strategy, N variables

for i, frame in enumerate(dfs):
    res = Trading_strat_2(dfs[i], bubs[i], hdwin, tranr, ma1, ma2, start, end, strat, stats)
    results[i, :, :] = res[0]
    

In [ ]:
df_p0 = pd.DataFrame(results[0, :, :])
df_p1 = pd.DataFrame(results[1, :, :])
df_p2 = pd.DataFrame(results[2, :, :])
#df_p3 = pd.DataFrame(results[3, :, :])
#df_p4 = pd.DataFrame(results[4, :, :])
#df_p5 = pd.DataFrame(results[5, :, :])


if stats == 'transactions': #return count of transactions instead of gross
    variables = ['Trades', 'Net' , 'SR']
if stats == 'exposure': #return count of transactions instead of gross
    variables = ['Exp', 'Net' , 'SR']
if stats == 'both': #return count of transactions instead of gross
    variables = ['Exp', 'Trades' , 'SR']

frames = [df_p0, df_p1, df_p2]#, df_p3]#, df_p4]#, df_p5] 
df_results = pd.concat(frames, axis=1)
df_results.columns=variables*3

df_results = df_results.set_index([strategies])

df_results = df_results.iloc[[0,3, 6, 9, 12, 13]]
df_results

In [ ]:
# Print to latex
if stats == 'transactions': #return count of transactions instead of gross
    print(df_results.style.format(subset=['Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Trades'], precision=0, escape='Latex').to_latex())
if stats == 'exposure': #return count of transactions instead of gross
    print(df_results.style.format(subset=['Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Exp'], precision=0, escape='Latex').to_latex())
else:
    print(df_results.style.format(subset=['Gross','Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').to_latex())

### Different stocks

#### Bub estimates

In [ ]:
amzn_bub, amzn_gp= get_combined_bub_estimates('AMZN',volume_weighted, ma, start_dates, end_dates)
tsla_bub, tsla_gp= get_combined_bub_estimates('TSLA',volume_weighted, ma, start_dates, end_dates)
googl_bub, googl_gp= get_combined_bub_estimates('GOOGL',volume_weighted, ma, start_dates, end_dates)

In [ ]:
col = 'bubcpall'
colors = ['red', 'blue', 'orange', 'green', 'black']
start = start_dates[14]
end = end_dates[-1]
print(start, end)

fig, ax = plt.subplots(1, 1, figsize=(10, 6), constrained_layout=True)

# Smoothing
spar = 1500
spar2 = 15000
amzn_bubs = amzn_bub[amzn_bub.bubcpall<spar]
amzn_bubs = amzn_bubs[amzn_bub.bubcpall>-spar]
tsla_bubs = tsla_bub[tsla_bub.bubcpall<spar]
tsla_bubs = tsla_bubs[tsla_bub.bubcpall>-spar]
googl_bubs = googl_bub[googl_bub.bubcpall<spar]
googl_bubs = googl_bubs[googl_bub.bubcpall>-spar]


# X -Values
x1 = tsla_bubs['date']
x2 = amzn_bubs['date']
x3 = googl_bubs['date']


y1 = (tsla_bubs[col]).rolling(ma).mean()
y2 = (amzn_bubs[col]).rolling(ma).mean()
y3 = (googl_bubs[col]).rolling(ma).mean()

# Plot
ax.plot(x1, y1, label = f'TSLA')#, color=colors[1])
ax.plot(x2, y2, label = f'AMZN')#, color=colors[2])
ax.plot(x3, y3, label = f'GOOGL')#, color=colors[3])
    
# Axes, Labels, title etc 
ax.set_xlim([pd.to_datetime(start, format = '%Y-%m-%d'),pd.to_datetime(end, format = '%Y-%m-%d')])
ax.set_ylabel(r'Bubble estimate')
ax.set_title(r'$\hat{\Pi}_{cp}(\tau)$', loc='center', fontsize='medium')
ax.legend(loc='upper left')
#ax.set_ylim(-60,200)

filepath = Path(f'{pf}/data/figures/bubbles_stocks') # Set name
plt.savefig(filepath) #bbox_inches='tight'
    
#ax.set_ylim(0, 100)

In [ ]:
start = start_dates[0]
end = end_dates[-1]
print(start, end)
names = ['TSLA', 'AMZN', 'GOOGL']
indices = pd.DataFrame()

for i, name in enumerate(names):
    # Read data
    sout = pd.read_csv (Path(f'{pf}/data/input/stockprices_{name}.csv')) # Data
    sout[name] = sout['snp']
    sout['date'] =  pd.to_datetime(sout['date'], format='%Y-%m-%d')
    sout = sout[sout['date'].between(start, end)]
    
    if i == 1:
        indices['date']=sout['date']
    
    # For calculating cov
    indices[name]=sout[name]
    


In [ ]:
s1= pd.read_csv (Path(f'{pf}/data/input/stockprices_TSLA.csv')) # Data

s2= pd.read_csv (Path(f'{pf}/data/input/stockprices_AMZN.csv')) # Data
s3= pd.read_csv (Path(f'{pf}/data/input/stockprices_GOOGL.csv')) # Data

s1['TSLA'] = s1['snp']
s2['AMZN'] = s2['snp']
s3['GOOGL'] = s3['snp']
#df = pd.merge(left=df, right=df_trate_1mo3mo, left_on="date", right_on="date", how="left")

sout = pd.merge(left=s1, right=s2, left_on='date', right_on='date', how='left')
sout = pd.merge(left=sout, right=s3, left_on='date', right_on='date', how='left')
sout

#### Trading on different stocks

In [ ]:
dfs = [tsla_bub, amzn_bub, googl_bub]
bubs = [tsla_gp, amzn_gp, googl_gp]

stats = 'exposure'

start = '2011-01-01'
end = '2021-12-31'
ma1= 63
ma2 = 252
tranr = 0.005
hdwin = 5
strat = 1

frames = []

results = np.empty((len(dfs), len(strategies), len(variables))) # number of different parameters, N strategy, N variables

for i, frame in enumerate(dfs):
    res = Trading_strat_2(dfs[i], bubs[i], hdwin, tranr, ma1, ma2, start, end, strat, stats)
    results[i, :, :] = res[0]
    
df_p0 = pd.DataFrame(results[0, :, :])
df_p1 = pd.DataFrame(results[1, :, :])
df_p2 = pd.DataFrame(results[2, :, :])
#df_p3 = pd.DataFrame(results[3, :, :])
#df_p4 = pd.DataFrame(results[4, :, :])
#df_p5 = pd.DataFrame(results[5, :, :])


if stats == 'transactions': #return count of transactions instead of gross
    variables = ['Trades', 'Net' , 'SR']
if stats == 'exposure': #return count of transactions instead of gross
    variables = ['Exp', 'Net' , 'SR']
if stats == 'both': #return count of transactions instead of gross
    variables = ['Exp', 'Trades' , 'SR']

frames = [df_p0, df_p1, df_p2]#, df_p3]#, df_p4]#, df_p5] 
df_results = pd.concat(frames, axis=1)
df_results.columns=variables*3

df_results = df_results.set_index([strategies])

df_results = df_results.iloc[[0,3, 6, 9, 12, 13]]
df_results

In [ ]:
# Print to latex
if stats == 'transactions': #return count of transactions instead of gross
    print(df_results.style.format(subset=['Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Trades'], precision=0, escape='Latex').to_latex())
if stats == 'exposure': #return count of transactions instead of gross
    print(df_results.style.format(subset=['Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').format(subset=['Exp'], precision=0, escape='Latex').to_latex())
else:
    print(df_results.style.format(subset=['Gross','Net'], precision=1, escape='Latex').format(subset=['SR'], precision=2, escape='Latex').to_latex())

In [ ]:
names = ['TSLA', 'AMZN', 'GOOGL']
colors = ['blue', 'red', 'orange', 'green']
labels = ['TSLA', 'AMZN', 'GOOGL']

start = start_dates[14]
end = end_dates[-1]
print(start, end)

fig, ax = plt.subplots(1, 1, figsize=(10, 6), constrained_layout=True)

indices = pd.DataFrame()

for i, name in enumerate(names):
    # Read data
    sout = pd.read_csv (Path(f'{pf}/data/input/stockprices_{name}.csv')) # Data
    sout[name] = sout['snp']
    sout['date'] =  pd.to_datetime(sout['date'], format='%Y-%m-%d')
    sout = sout[sout['date'].between(start, end)]
    
    # X -Values
    x = sout['date']

    #Y - Values 
    y = sout[name]/sout[name].iloc[0]*100
    
    # Plot
    ax.plot(x, y, label = f'{labels[i]}', color=colors[i])
    
    # For calculating cov
    indices[name]=sout[name]

# Axes, Labels, title etc 
ax.set_xlim([pd.to_datetime(start, format = '%Y-%m-%d'),pd.to_datetime(end, format = '%Y-%m-%d')])
ax.set_ylabel(r'Index, 1996-01-02 = 100')
ax.set_title(r'Stock price indices', loc='center', fontsize='medium')
ax.legend(loc='upper left')
    
filepath = Path(f'{pf}/data/figures/descriptive_stonks') # Set name
plt.savefig(filepath) #bbox_inches='tight'

corr = indices.corr()
corr
#ax.set_ylim(-10, 100)